In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

import random

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_torch(seed=666)

In [2]:
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'
FOLDS_PATH = '../input/split-k-folds/'
TRAIN_PATH = '../input/ranzcr-clip-catheter-line-classification/train'

target_size=11
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
             'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
             'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
             'Swan Ganz Catheter Present']

# cargamos los DATOS
dfx = pd.read_csv(FOLDS_PATH + "train_folds.csv") 
 
# filtramos. Por defecto dejo en "0" el fold_test
df_test_fold = dfx[dfx.fold_test == 0].reset_index(drop=True)
print(df_test_fold.shape)
df_test_fold['file_path'] = df_test_fold.StudyInstanceUID.apply(lambda x: os.path.join('../input/ranzcr-clip-catheter-line-classification/train', f'{x}.jpg'))
df_test_fold.head()

(1245, 15)


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,fold,fold_test,file_path
0,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814,0,0,../input/ranzcr-clip-catheter-line-classificat...
1,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1,0,0,../input/ranzcr-clip-catheter-line-classificat...
2,1.2.826.0.1.3680043.8.498.23522123195677570717...,0,0,0,0,0,0,0,1,0,0,0,20335f7d1,4,0,../input/ranzcr-clip-catheter-line-classificat...
3,1.2.826.0.1.3680043.8.498.96303608016739892857...,0,0,0,0,0,0,0,1,0,0,0,528f079f0,2,0,../input/ranzcr-clip-catheter-line-classificat...
4,1.2.826.0.1.3680043.8.498.30798392240062365831...,0,0,0,0,0,0,0,0,0,1,0,3c1fc81a3,2,0,../input/ranzcr-clip-catheter-line-classificat...


In [3]:
model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4'] * 3
enet_type  = [ 
#     'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 
#     'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention',
#     'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d',
    'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320',
    'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512',
    'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores'
]
model_path  = [
#         '../input/efficient-b5-chestx/efficientnet_b5_fold0_step0_best_loss_cpu.pth',
#         '../input/efficient-b5-chestx/efficientnet_b5_fold1_step0_best_loss_cpu.pth',
#          '../input/efficient-b5-chestx/efficientnet_b5_fold2_step0_best_loss_cpu.pth',
#          '../input/efficient-b5-chestx/efficientnet_b5_fold3_step0_best_loss_cpu.pth',
#          '../input/efficient-b5-chestx/efficientnet_b5_fold4_step0_best_loss_cpu.pth',
#         '../input/efficientb4attention/efficientnet-b4_fold0_step0_best_loss_cpu.pth',
#         '../input/efficientb4attention/efficientnet-b4_fold1_step0_best_loss.pth',
#         '../input/efficientb4attention/efficientnet-b4_fold2_step0_best_loss_cpu.pth',
#         '../input/efficientb4attention/efficientnet-b4_fold3_step0_best_loss.pth',
#         '../input/efficientb4attention/efficientnet-b4_fold4_step0_best_loss_cpu.pth',
#         '../input/resnext50-32x4d/resnext50_32x4d_fold0_step0_best_loss_cpu_12epochs.pth',
#         '../input/resnext50-32x4d/resnext50_32x4d_fold1_step0_best_loss_cpu_12epochs.pth',
#         '../input/resnext50-32x4d/resnext50_32x4d_fold2_step0_best_loss_12epochs.pth',
#         '../input/resnext50-32x4d/resnext50_32x4d_fold3_step0_best_loss_12epochs.pth',
#         '../input/resnext50-32x4d/resnext50_32x4d_fold4_step0_best_loss_12epochs.pth',   
        '../input/resnet200d-640/resnet200d_320_fold0_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold1_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold2_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold3_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold4_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold0_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold1_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold2_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold3_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold4_step0_best_loss_cpu.pth',
    '../input/mejores-modelos-encontrados/resnet200d_fold0_cv953.pth/resnet200d_fold0_cv953.pth',
      '../input/mejores-modelos-encontrados/resnet200d_fold1_cv955.pth/resnet200d_fold1_cv955.pth',
      '../input/mejores-modelos-encontrados/resnet200d_fold2_cv955.pth/resnet200d_fold2_cv955.pth',
      '../input/mejores-modelos-encontrados/resnet200d_fold3_cv957.pth/resnet200d_fold3_cv957.pth',
      '../input/mejores-modelos-encontrados/resnet200d_fold4_cv954.pth/resnet200d_fold4_cv954.pth'
]

In [4]:
# model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4']

# enet_type  = ['resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320']

# model_path = [
#     '../input/resnet200d-640/resnet200d_320_fold0_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold1_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold2_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold3_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold4_step0_best_loss_cpu.pth'
# ]

In [5]:
# calculando pesos a partir del score
def calculate_weigths_inverse(lista_scores):
    
    lista_scores = np.array(lista_scores)

    weights = 100 - lista_scores
    total_score = np.sum(weights)

    weights = weights/total_score

    return weights

Guardo todo en una lista

In [6]:
input_folds = []
for i in range(len(enet_type)):
    preds = pd.read_csv('../input/buscador-de-pesos-ranzcr/' + enet_type[i] + '_' + model_names[i] + '.csv')
    input_folds.append(preds)

Creo un modelo por cada label y, luego, calculo la media de todos para asignar el peso final

In [7]:
input_folds[0][['ETT - Normal']]

,ETT - Normal
0,0.000078
1,0.001692
2,0.000003
3,0.000015
4,0.000006
...,...
1240,0.000002
1241,0.000002
1242,0.000019
1243,0.976278


In [8]:
def get_coeficients_normalized(coef):
    suma = np.sum(coef)
    return coef / suma

def save_csv(target, coef):
    names = [a + '_' + b for a, b in zip(enet_type, model_names)]
    df_model_coef = pd.DataFrame(
    {'name': names,
     'weigth': coef
    })
    df_model_coef.to_csv(target + '_weigths.csv', index=False)
    df_model_coef

In [9]:

coeficientes = []
for i in range(len(target_cols)):
    # voy columna/target por columna
    target = target_cols[i]
    print('Searching weigths for ' + target)
    # pillo las variables 
    cols_for_model = []
    for j in range(len(enet_type)):
        col = input_folds[j][[target]]
        # lo paso a 0's o 1's
#         col = (col >= 0.5).astype(int)
        cols_for_model.append(col)
    # creo el modelo
    X = np.column_stack(cols_for_model)
    y = df_test_fold[[target]].values.ravel()
    reg = RandomForestClassifier().fit(X, y)
    print('Score obtenido: ' + str(round(reg.score(X, y), 2)))
    reg_coef = reg.feature_importances_
    best_model_index = np.argmax(reg_coef)
    worst_model_index = np.argmin(reg_coef)
    print('El modelo con más peso es: ' + enet_type[best_model_index] + '_' + model_names[best_model_index])
    print('El modelo con menos peso es: ' + enet_type[worst_model_index] + '_' + model_names[worst_model_index])
    print(reg_coef)
    print(np.sum(reg_coef))
    coeficientes.append(reg_coef) # coef_
#     coef_normalized = get_coeficients_normalized(reg_coef.ravel()) # solo en caso de no ser randomforest
    save_csv(target, reg_coef)
    print('---------------------------------------------------------------------------')

Searching weigths for ETT - Abnormal
Score obtenido: 1.0
El modelo con más peso es: resnet200d_Mejores_fold2
El modelo con menos peso es: resnet200d_320_512_fold4
[0.06669317 0.04302447 0.05484585 0.04310739 0.06364138 0.04012328
 0.0439434  0.05445216 0.02628187 0.02136336 0.08331932 0.08865663
 0.14907749 0.13081801 0.09065222]
1.0
---------------------------------------------------------------------------
Searching weigths for ETT - Borderline
Score obtenido: 1.0
El modelo con más peso es: resnet200d_Mejores_fold0
El modelo con menos peso es: resnet200d_320_fold4
[0.02033815 0.0313666  0.03051545 0.03333222 0.01461991 0.02667525
 0.02885898 0.0630181  0.05435063 0.01649515 0.17929778 0.12308516
 0.12709617 0.1029055  0.14804495]
1.0
---------------------------------------------------------------------------
Searching weigths for ETT - Normal
Score obtenido: 1.0
El modelo con más peso es: resnet200d_Mejores_fold4
El modelo con menos peso es: resnet200d_320_fold0
[0.00412671 0.0142764

In [10]:
# aqui hago la media de coeficientes y tiro palante
final_coefs = np.mean(coeficientes, axis=0).ravel()
best_model_index = np.argmax(final_coefs)
worst_model_index = np.argmin(final_coefs)
print('El modelo con más peso es: ' + enet_type[best_model_index] + '_' + model_names[best_model_index])
print('El modelo con menos peso es: ' + enet_type[worst_model_index] + '_' + model_names[worst_model_index])
print(final_coefs)

El modelo con más peso es: resnet200d_Mejores_fold2
El modelo con menos peso es: resnet200d_320_fold2
[0.02514347 0.02590547 0.021609   0.03677462 0.0245595  0.04652643
 0.05295085 0.0488313  0.0367901  0.03782393 0.14568709 0.10900348
 0.15281534 0.11501464 0.12056477]


Normalizo para que la suma valga 1

In [11]:
suma = np.sum(final_coefs)
print(suma)
final_coefs = final_coefs / suma
print(np.sum(final_coefs))

1.0
1.0


In [12]:
# guardo
# save in a csv
names = [a + '_' + b for a, b in zip(enet_type, model_names)]
df_model_coef = pd.DataFrame(
    {'name': names,
     'weigth': final_coefs
    })
df_model_coef.to_csv('models_weigths.csv', index=False)
df_model_coef.sort_values(by=['weigth'], ascending=False)

,name,weigth
12,resnet200d_Mejores_fold2,0.152815
10,resnet200d_Mejores_fold0,0.145687
14,resnet200d_Mejores_fold4,0.120565
13,resnet200d_Mejores_fold3,0.115015
11,resnet200d_Mejores_fold1,0.109003
6,resnet200d_320_512_fold1,0.052951
7,resnet200d_320_512_fold2,0.048831
5,resnet200d_320_512_fold0,0.046526
9,resnet200d_320_512_fold4,0.037824
8,resnet200d_320_512_fold3,0.036790


In [13]:
# x = np.random.rand(10, 25)
# a = df_model_coef[['weigth']].values.flatten()

# np.average(x, axis=1, weights = a)

ACCURACIES

In [14]:
# funcioens un poco cambiadas para este notebook
def assign_weights_by_target(preds):
    final_preds = np.zeros((preds[0].shape[0], 11))
    # recorro las columnas
    for i in range(len(target_cols)):
        target = target_cols[i]
        target_w = pd.read_csv('./' + target + '_weigths.csv').values
        # recorro los modelos
        for j in range(len(enet_type)):
            # preds por target
            model_preds = preds[j]
            model_preds_target = model_preds[:, i]
            # weigth
            weigth = target_w[j, 1]
            model_preds_target = model_preds_target * weigth
            final_preds[:, i] += model_preds_target
    
    return final_preds

def assign_weigths(preds):
    final_preds = np.zeros((preds[0].shape[0], 11))
    weigths = final_coefs
    for j in range(len(enet_type)):
        model_preds = preds[j]
        model_preds = model_preds * weigths[j]
        final_preds += model_preds
        
    return final_preds

In [15]:
test_preds = []
for i in range(len(enet_type)):
    preds = pd.read_csv('../input/buscador-de-pesos-ranzcr/' + enet_type[i] + '_' + model_names[i] + '.csv')
    preds = preds.iloc[:, 1:12].values
    test_preds.append(preds)

In [16]:
from sklearn.metrics import roc_auc_score

def get_auc(y_pred, y_real):
    acc_fold = np.mean([roc_auc_score(y_real[:, i], y_pred[:, i]) for i in range(11)])

    print("Final ACC for this fold:", acc_fold)
    
y_real = df_test_fold.iloc[:, 1:12].values
    
# media
get_auc(np.mean(test_preds, axis = 0), y_real)

# pesos generales
y_preds = assign_weigths(test_preds)
get_auc(y_preds, y_real)

# pesos por target
y_preds = assign_weights_by_target(test_preds)
get_auc(y_preds, y_real)

Final ACC for this fold: 0.9716265745920509
Final ACC for this fold: 0.9793306403094177
Final ACC for this fold: 0.9807009183127575
